<a href="https://colab.research.google.com/github/mirkhalilrehman/Image-Segmentation-UNet-Pytorch/blob/main/UNet_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#To be continued not completed

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile

# Specify the path to the zip file in your Google Drive
zip_file_path = '/content/drive/MyDrive/lungs.zip'

# Specify the directory where you want to extract the files
extract_dir = '/content/drive/MyDrive/lungs/'

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Files extracted to {extract_dir}")


KeyboardInterrupt: 

In [ ]:
os.listdir('/content/drive/MyDrive/lungs/')

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import Dataset,DataLoader
import matplotlib.pyplot as plt
import os
from PIL import Image


In [20]:
class UNet(nn.Module):
  def __init__(self,in_channels,out_channels):
    super(UNet,self).__init__()
    self.enc1 = self.conv_block(in_channels, 64)
    self.enc2 = self.conv_block(64, 128)
    self.enc3 = self.conv_block(128, 256)
    self.enc4 = self.conv_block(256, 512)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    self.upconv3=nn.ConvTranspose2d(512,256,kernel_size=2, stride=2)
    self.upconv2=nn.ConvTranspose2d(256,128,kernel_size=2, stride=2)
    self.upconv1=nn.ConvTranspose2d(128,64,kernel_size=2, stride=2)

    self.dec1=self.conv_block(512,256)
    self.dec2=self.conv_block(256,128)
    self.dec3=self.conv_block(128,64)
    self.out=nn.Conv2d(64,out_channels=out_channels,kernel_size=1)

  def conv_block(self,in_channels,out_channels):
    return nn.Sequential([
        nn.Conv2d(in_channels,out_channels,kernel_size=3,padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
         nn.Conv2d(out_channels,out_channels,kernel_size=3,padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    ])

  def forward(self,x):
    x1=self.enc1(x)
    x2=self.enc2(self.pool(x1))
    x3=self.enc3(self.pool(x2))
    x4=self.enc4(self.pool(x3))

    x=self.upconv3(x4)
    x=torch.cat([x,x3],dim=1)
    x=self.dec1(x)

    x=self.upconv2(x)
    x=torch.cat([x,x2],dim=1)
    x=self.dec2(x)

    x=self.upconv1(x)
    x=torch.cat([x,x1],dim=1)
    x=self.dec3(x)

    return self.out(x)



In [23]:
class LungSegmentation(Dataset):
    def __init__(self, image_paths, mask_paths, image_transform=None, mask_transform=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.image_filenames = sorted(os.listdir(image_paths))
        self.mask_filenames = sorted(os.listdir(mask_paths))
        self.image_transform = image_transform
        self.mask_transform = mask_transform

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_dir = os.path.join(self.image_paths, self.image_filenames[idx])
        mask_dir = os.path.join(self.mask_paths, self.mask_filenames[idx])

        image = Image.open(image_dir).convert('RGB')
        mask = Image.open(mask_dir).convert('L')

        if self.image_transform:
            image = self.image_transform(image)
        if self.mask_transform:
            mask = self.mask_transform(mask)

        mask = mask.squeeze(0)
        mask = torch.where(mask > 0, 1, 0).float()

        return image, mask

In [26]:
image_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

mask_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])


In [27]:
images_path = "/content/drive/MyDrive/lungs/CXR_png"
masks_path = "/content/drive/MyDrive/lungs/masks"

In [28]:
dataset = LungSegmentation(images_path, masks_path, image_transform=image_transform, mask_transform=mask_transform)


In [30]:
from sklearn.model_selection import train_test_split

In [32]:
len(dataset)

800

In [34]:
import os

# Define the directories
image_dir = '/content/drive/MyDrive/lungs/CXR_png'
mask_dir = '/content/drive/MyDrive/lungs/masks'

# List the filenames in both directories
def remove_extension(filename):
    return os.path.splitext(filename)[0]

# Remove extensions and get base names for both images and masks
image_basenames = [remove_extension(f) for f in image_filenames]
mask_basenames = [remove_extension(f) for f in mask_filenames]

# Check for any filenames in one list that are missing in the other
missing_images = [mask for mask in mask_basenames if mask not in image_basenames]
missing_masks = [image for image in image_basenames if image not in mask_basenames]

# Print the results
if missing_images:
    print("Missing images for the following masks:")
    print(missing_images)
else:
    print("No missing images.")

if missing_masks:
    print("Missing masks for the following images:")
    print(missing_masks)
else:
    print("No missing masks.")


Missing images for the following masks:
['CHNCXR_0001_0_mask', 'CHNCXR_0002_0_mask', 'CHNCXR_0003_0_mask', 'CHNCXR_0004_0_mask', 'CHNCXR_0005_0_mask', 'CHNCXR_0006_0_mask', 'CHNCXR_0007_0_mask', 'CHNCXR_0008_0_mask', 'CHNCXR_0009_0_mask', 'CHNCXR_0010_0_mask', 'CHNCXR_0011_0_mask', 'CHNCXR_0012_0_mask', 'CHNCXR_0013_0_mask', 'CHNCXR_0014_0_mask', 'CHNCXR_0015_0_mask', 'CHNCXR_0016_0_mask', 'CHNCXR_0017_0_mask', 'CHNCXR_0018_0_mask', 'CHNCXR_0019_0_mask', 'CHNCXR_0020_0_mask', 'CHNCXR_0021_0_mask', 'CHNCXR_0022_0_mask', 'CHNCXR_0023_0_mask', 'CHNCXR_0024_0_mask', 'CHNCXR_0026_0_mask', 'CHNCXR_0027_0_mask', 'CHNCXR_0028_0_mask', 'CHNCXR_0029_0_mask', 'CHNCXR_0030_0_mask', 'CHNCXR_0031_0_mask', 'CHNCXR_0032_0_mask', 'CHNCXR_0033_0_mask', 'CHNCXR_0034_0_mask', 'CHNCXR_0035_0_mask', 'CHNCXR_0041_0_mask', 'CHNCXR_0042_0_mask', 'CHNCXR_0043_0_mask', 'CHNCXR_0044_0_mask', 'CHNCXR_0045_0_mask', 'CHNCXR_0046_0_mask', 'CHNCXR_0047_0_mask', 'CHNCXR_0048_0_mask', 'CHNCXR_0049_0_mask', 'CHNCXR_0050_

In [36]:
import os


# Get all image filenames (without extension)
image_files = [f.split('.')[0] for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

# Check for missing masks
missing_masks = []
for image_name in image_files:
    mask_name = image_name + '_mask.png'  # Assuming the mask is a PNG file and follows this pattern
    if not os.path.exists(os.path.join(mask_dir, mask_name)):
        missing_masks.append(image_name)

# Print the results
if missing_masks:
    print(f'Missing masks for the following images: {missing_masks}')
else:
    print('No masks are missing.')


Missing masks for the following images: ['CHNCXR_0025_0', 'CHNCXR_0036_0', 'CHNCXR_0037_0', 'CHNCXR_0038_0', 'CHNCXR_0039_0', 'CHNCXR_0040_0', 'CHNCXR_0065_0', 'CHNCXR_0181_0', 'CHNCXR_0182_0', 'CHNCXR_0183_0', 'CHNCXR_0184_0', 'CHNCXR_0185_0', 'CHNCXR_0186_0', 'CHNCXR_0187_0', 'CHNCXR_0188_0', 'CHNCXR_0189_0', 'CHNCXR_0190_0', 'CHNCXR_0191_0', 'CHNCXR_0192_0', 'CHNCXR_0193_0', 'CHNCXR_0194_0', 'CHNCXR_0195_0', 'CHNCXR_0196_0', 'CHNCXR_0197_0', 'CHNCXR_0198_0', 'CHNCXR_0199_0', 'CHNCXR_0200_0', 'CHNCXR_0201_0', 'CHNCXR_0202_0', 'CHNCXR_0203_0', 'CHNCXR_0204_0', 'CHNCXR_0205_0', 'CHNCXR_0206_0', 'CHNCXR_0207_0', 'CHNCXR_0208_0', 'CHNCXR_0209_0', 'CHNCXR_0210_0', 'CHNCXR_0211_0', 'CHNCXR_0212_0', 'CHNCXR_0213_0', 'CHNCXR_0214_0', 'CHNCXR_0215_0', 'CHNCXR_0216_0', 'CHNCXR_0217_0', 'CHNCXR_0218_0', 'CHNCXR_0219_0', 'CHNCXR_0220_0', 'CHNCXR_0336_1', 'CHNCXR_0341_1', 'CHNCXR_0342_1', 'CHNCXR_0343_1', 'CHNCXR_0344_1', 'CHNCXR_0345_1', 'CHNCXR_0346_1', 'CHNCXR_0347_1', 'CHNCXR_0348_1', 'CHNCXR

In [31]:
train_ds,test_ds=train_test_split(dataset,test_size=0.2,random_state=42)
train_ds,val_ds=train_test_split(train_ds,test_size=0.2,random_state=42)

IndexError: list index out of range

In [29]:
from sklearn import train_test_split
# Split data into training and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = t.utils.data.random_split(dataset, [train_size, val_size])

dataloader_train = t.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True)
dataloader_val = t.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False)

ImportError: cannot import name 'train_test_split' from 'sklearn' (/usr/local/lib/python3.11/dist-packages/sklearn/__init__.py)